<a href="https://colab.research.google.com/github/MASAYA-KINOSHITA/first-Repository/blob/main/Software_anomaly_detection_%E7%9B%B8%E9%96%A2%E4%BF%82%E6%95%B026_stacking_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ライブラリのインポート
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用
import seaborn as sns; sns.set() # グラフ描画用
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)     
from sklearn.metrics import roc_auc_score # モデル評価用(auc)

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
#メモリ節約用の関数
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# data_train csvファイルの読み込み
train_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/train.zip (Unzipped Files)/train.csv',usecols=['Anomaly',
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [4]:
# data_test csvファイルの読み込み
test_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/test.zip (Unzipped Files)/test.csv',usecols=[
 'GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))',
 'Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize))',
 'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
 'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
 'Process CPU : (MXBean(java.lang:type=OperatingSystem).ProcessCpuLoad)',
 'Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad)',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Process CPU : (\\Process(java)\\CPU)',
 'Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max))',
 'Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
 'Prepared statement cache hit rate : ((MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheHitCount / MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).PrepStmtCacheMissCount))',
 'Memory space usage : ((MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Perm Gen,type=MemoryPool).Usage.max))',
 'Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))',
 'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
 'host',
 'Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)',
 'Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max))',
 'Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize))',
 'GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))',
 'Rel. Swap Usage : ((\\Swap\\used / \\Swap\\total))' 
 ])

In [5]:
#スコア0.9279の予測データを読み込み
#test_y = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/xgboost_trained_model_26_param_tuning_test_reloaded2.csv')

In [6]:
#目的変数のみを取り出し値を四捨五入
#test_y=test_y["Anomaly"]
#test_y=test_y.round(0).astype(int)

In [7]:
#訓練データから目的変数のみを取り出す
train_y=train_df["Anomaly"]

In [8]:
#訓練データから目的変数を除去
train_x=train_df.drop(["Anomaly"],axis=1)
test_x=test_df

In [9]:
#全データをpandasに適用
train_x = pd.DataFrame(train_x)
test_x = pd.DataFrame(test_x)
train_y = pd.Series(train_y)
#test_y = pd.Series(test_y)

In [10]:
#trainデータとtestデータの予測値を結合
#train_x = pd.concat([train_x, test_x],axis=0)
#train_y = pd.concat([train_y, test_y],axis=0)

In [11]:
# 学習データにget_dummiesによるone-hot encodingを行う
train_x = pd.get_dummies(train_x, columns=["host"])
test_x = pd.get_dummies(test_x, columns=["host"])

In [12]:
'''
reduce_mem_usage(train_x, verbose=True)
reduce_mem_usage(test_x, verbose=True)
reduce_mem_usage(test_df, verbose=True)
reduce_mem_usage(train_df, verbose=True)
'''

Mem. usage decreased to 313.28 Mb (67.9% reduction)
Mem. usage decreased to 173.08 Mb (70.5% reduction)
Mem. usage decreased to 167.50 Mb (71.2% reduction)
Mem. usage decreased to 102.71 Mb (71.8% reduction)


,host,Anomaly,Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount),"GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))","GC time : (incld/dx (MXBean(java.lang:name=PS MarkSweep,type=GarbageCollector).CollectionTime))","GC time : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionTime))","Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.used / MXBean(java.lang:name=PS Eden Space,type=MemoryPool).Usage.max))","Memory space usage : ((MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.committed / MXBean(java.lang:name=PS Old Gen,type=MemoryPool).Usage.max))",...,Rel. heap committed : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.committed / MXBean(java.lang:type=Memory).HeapMemoryUsage.max)),Rel. heap usage : ((MXBean(java.lang:type=Memory).HeapMemoryUsage.used / MXBean(java.lang:type=Memory).HeapMemoryUsage.max)),Rel. nonHeap usage : ((MXBean(java.lang:type=Memory).NonHeapMemoryUsage.used / MXBean(java.lang:type=Memory).NonHeapMemoryUsage.max)),Rel. open file descriptors : ((MXBean(java.lang:type=OperatingSystem).OpenFileDescriptorCount / MXBean(java.lang:type=OperatingSystem).MaxFileDescriptorCount)),Rel. physical mem usage : ((MXBean(java.lang:type=OperatingSystem).FreePhysicalMemorySize / MXBean(java.lang:type=OperatingSystem).TotalPhysicalMemorySize)),Rel. swap usage : ((MXBean(java.lang:type=OperatingSystem).FreeSwapSpaceSize / MXBean(java.lang:type=OperatingSystem).TotalSwapSpaceSize)),"Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)","Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)",System CPU : (MXBean(java.lang:type=OperatingSystem).SystemCpuLoad),Total thread count : (MXBean(java.lang:type=Threading).ThreadCount)
0,lphost06,0,62.0,0.0,0.0,0.0,0.927246,0.271729,0.165527,0.468262,...,0.456787,0.345703,0.829590,0.142578,0.682617,1.000000,0.0,0.0,0.004002,78.0
1,lphost06,0,62.0,0.0,0.0,0.0,0.927246,0.271729,0.178833,0.468262,...,0.456787,0.350342,0.829590,0.142578,0.682617,1.000000,0.0,0.0,0.004002,78.0
2,lphost06,0,62.0,0.0,0.0,0.0,0.927246,0.271729,0.196533,0.468262,...,0.456787,0.356689,0.829590,0.142578,0.682617,1.000000,0.0,0.0,0.005001,78.0
3,lphost06,0,62.0,0.0,0.0,0.0,0.927246,0.271729,0.209961,0.468262,...,0.456787,0.361572,0.829590,0.142578,0.682617,1.000000,0.0,0.0,0.004002,78.0
4,lphost06,0,62.0,0.0,0.0,0.0,0.927246,0.271729,0.222900,0.468262,...,0.456787,0.365967,0.829590,0.142578,0.683105,1.000000,0.0,0.0,0.004002,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765536,lphost18,0,36.0,0.0,0.0,0.0,0.335938,0.539551,0.215210,0.055176,...,0.242432,0.107483,0.588867,0.133545,0.617188,0.999512,0.0,0.0,0.004002,44.0
1765537,lphost18,0,36.0,0.0,0.0,0.0,0.335938,0.539551,0.228760,0.055176,...,0.242432,0.111816,0.588867,0.133545,0.617188,0.999512,0.0,0.0,0.003000,44.0
1765538,lphost18,0,36.0,0.0,0.0,0.0,0.335938,0.539551,0.241699,0.055176,...,0.242432,0.116821,0.588867,0.133545,0.617188,0.999512,0.0,0.0,0.003000,44.0
1765539,lphost18,0,36.0,0.0,0.0,0.0,0.335938,0.539551,0.256592,0.055176,...,0.242432,0.121704,0.588867,0.133545,0.617188,0.999512,0.0,0.0,0.004002,44.0


使用するパラメータ

{'batch_norm': 'no', 'batch_size': 32.0, 'hidden_activation': 'prelu', 'hidden_dropout': 0.2, 'hidden_layers': 3.0, 'hidden_units': 224.0, 'input_dropout': 0.0, 'optimizer': {'lr': 1.7231851937742915e-05, 'type': 'sgd'}

In [12]:
# tensorflowの警告抑制
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# -----------------------------------
# ニューラルネットの実装
# -----------------------------------
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import ReLU, PReLU
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD,Adam
from sklearn.preprocessing import StandardScaler

#ハイパーパラメータ
batch_norm = 'no', 
batch_size = 32
hidden_activation = 'PReLU', 
hidden_dropout = 0.2, 
hidden_layers = 3.0
hidden_units = 224 
input_dropout = 0.0,
optimizer_lr = 0.000017231851937742915

In [13]:
import xgboost as xgb

params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.1,
    'gamma': 0.00014307741815837628,
    'alpha': 0.0,
    'lambda': 1.0,
    'min_child_weight': 1.4034295969588149,
    'max_depth': 9,
    'subsample': 0.95,
    'colsample_bytree': 0.95,
    'random_state': 71,
}

In [22]:
# sklearn.linear_model.LinearRegression クラスを読み込み
from sklearn import linear_model
# スタッキング
from sklearn.model_selection import KFold
# 各クラスは、fitで学習し、predictで予測値の確率を出力する
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数

def predict_cv_nn(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        
        # データのスケーリング      
        scaler = StandardScaler()
        tr_x = scaler.fit_transform(tr_x)
        va_x = scaler.transform(va_x)
        test_x = scaler.transform(test_x)
        # ニューラルネットモデルの構築
        model.add(Dense(224, activation='PReLU', input_shape=(tr_x.shape[1],)))
        model.add(Dropout(0.2))
        model.add(Dense(224, activation='PReLU'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
        # オプティマイザ
        optimizer = SGD(lr=optimizer_lr, decay=1e-6, momentum=0.9, nesterov=True)
        
        # 目的関数、評価指標などの設定 
        model.compile(loss='binary_crossentropy',
                      optimizer=optimizer, metrics=['accuracy'])

        # エポック数、アーリーストッピング
        # あまりepochを大きくすると、小さい学習率のときに終わらないことがあるので注意
        nb_epoch = 30
        patience = 10
        early_stopping = EarlyStopping(patience=patience, restore_best_weights=True)
        model.fit(tr_x, tr_y,
                  epochs=nb_epoch,
                  batch_size=batch_size, verbose=1,
                  validation_data=(va_x, va_y),
                  callbacks=[early_stopping])
        
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)
    return pred_train, preds_test


def predict_cv_xgb(train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    # 特徴量と目的変数をxgboostのデータ構造に変換する
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dtest = xgb.DMatrix(test_x)
        dvalid = xgb.DMatrix(va_x,va_y)
        dvalid_x = xgb.DMatrix(va_x)

        num_round = 50000
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        model = xgb.train(params, dtrain, num_round, evals=watchlist,
                            early_stopping_rounds=20)
       
        pred = model.predict(dvalid_x)
        preds.append(pred)
        pred_test = model.predict(dtest)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)
    return pred_train, preds_test


def predict_cv_linear(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        
        model.fit(tr_x, tr_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)
    return pred_train, preds_test

# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model1_nn = Sequential()
pred_train_1a, pred_test_1a = predict_cv_nn(model1_nn, train_x, train_y, test_x)
pred_train_1a=np.ravel(pred_train_1a)
pred_test_1a=np.ravel(pred_test_1a)

# model1_xgb
pred_train_1b, pred_test_1b = predict_cv_xgb(train_x, train_y, test_x)

# 1層目のモデルの評価
print(f'logloss: {log_loss(train_y, pred_train_1a, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1b, eps=1e-7):.4f}')

# 予測値を特徴量としてデータフレームを作成
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b})
test_x_2 = pd.DataFrame({'pred_1a': pred_test_1a, 'pred_1b': pred_test_1b})

# 2層目のモデル
# pred_train_2は、2層目のモデルの学習データのクロスバリデーションでの予測値
# pred_test_2は、2層目のモデルのテストデータの予測値
model_2 = linear_model.LinearRegression()
pred_train_2, pred_test_2 = predict_cv_linear(model_2, train_x_2, train_y, test_x_2)
print(f'logloss: {log_loss(train_y, pred_train_2, eps=1e-7):.4f}')

Epoch 1/30
41380/41380 [==============================] - 129s 3ms/step - loss: 0.1160 - accuracy: 0.9781 - val_loss: 0.0561 - val_accuracy: 0.9858
Epoch 2/30
41380/41380 [==============================] - 128s 3ms/step - loss: 0.0497 - accuracy: 0.9865 - val_loss: 0.0380 - val_accuracy: 0.9930
Epoch 3/30
41380/41380 [==============================] - 127s 3ms/step - loss: 0.0383 - accuracy: 0.9897 - val_loss: 0.0314 - val_accuracy: 0.9934
Epoch 4/30
41380/41380 [==============================] - 128s 3ms/step - loss: 0.0331 - accuracy: 0.9913 - val_loss: 0.0280 - val_accuracy: 0.9934
Epoch 5/30
41380/41380 [==============================] - 127s 3ms/step - loss: 0.0301 - accuracy: 0.9921 - val_loss: 0.0258 - val_accuracy: 0.9933
Epoch 6/30
41380/41380 [==============================] - 128s 3ms/step - loss: 0.0279 - accuracy: 0.9926 - val_loss: 0.0241 - val_accuracy: 0.9933
Epoch 7/30
41380/41380 [==============================] - 129s 3ms/step - loss: 0.0264 - accuracy: 0.9928 - val_

In [23]:
# submission csvファイルの読み込み
submission_df = pd.read_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/data.zip (Unzipped Files)/sample_submission.csv')

In [24]:
id_df=submission_df.iloc[:,0]
id_df.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [25]:
pred_df = pd.DataFrame(pred_test_2)
pred_df.head()

,0
0,-0.000001
1,-0.000001
2,-0.000001
3,-0.000001
4,-0.000001


In [26]:
output_df=pd.concat([id_df, pred_df],axis=1)
output_df.columns=['id','Anomaly']
output_df.head()

,id,Anomaly
0,0,-0.000001
1,1,-0.000001
2,2,-0.000001
3,3,-0.000001
4,4,-0.000001


In [27]:
output_df.to_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/stacking_26_param_tuning_test_reloaded4.csv',index=False)

In [21]:
train_x_2.head()

,pred_1a,pred_1b
0,0.002273,0.000005
1,0.003335,0.000028
2,0.003206,0.000019
3,0.000413,0.000034
4,0.003206,0.000019


In [24]:
#train_x_2.to_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/stacking_train_x_2.csv',index=False)

In [25]:
#test_x_2.to_csv('/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/stacking_test_x_2.csv',index=False)